In [ ]:
# coding: utf8

#Import

In [ ]:
import os
import sys
import json
import pprint
from pyes import *
from datetime import date, datetime

#Confinguration

#Class project

In [ ]:
class Demo(object):
    """
    """
    
    def __init__(self, title='title of the demo', desc='...', Type='bonnus', date=date.today(), html=u'index.html'):
        self._id = ''
        
        self.title = title
        self.date = date
        self.source = html
        self.demoType = Type
        self.description = desc
        
        self.tools = []
        self.credits = []
        self.seeAlso = []
        self.score = 1.0
    
    
    @classmethod
    def fetch(self, idx, elastic=ES('127.0.0.1:9200')):
        doc = elastic.get('lmd', 'demo', idx)
        
        d = Demo(doc.title, doc.description, doc.Type, datetime.strptime(doc.date,"%Y-%m-%d"), doc.source)
        
        d.addTools(doc.tools)
        d.addCredits(doc.credits)
        #d.seeAlso(doc.seeAlso)
        
        d._id = idx
        
        return d
    
    def __addAttributes(self, attr, values):
        for v in values:
            if v not in attr: attr.append(v)
                
    def addTools(self, values):
        self.__addAttributes(self.tools, values)
        
    def addCredits(self, values):
        self.__addAttributes(self.credits, values)
    
    def addSeeAlso(self, values):
        self.__addAttributes(self.seeAlso, values)
                
    @property
    def show(self):
        return {u'title': self.title,
           u'description': self.description,
           u'type': self.demoType,
           u'date': self.date.strftime(u'%Y-%m-%d'),
           u'source': self.source,
           u'tools': self.tools,
           u'credits': self.credits,
           u'seeAlso': self.seeAlso
          }


#Loader

### connection

In [ ]:
esConn = ES('127.0.0.1:9200')

In [ ]:
#add mapping
mapping = {}

In [ ]:
d = Demo(title='title of the demo N°2', Type='code')

In [ ]:
d = Demo(title=u'2012/2013 Paris car accidents', Type=u'graph', html=u'cars_one.html')

d.description = u'Which types of vehicule are involved in accidents for this period?'

d.addTools([u'Spark',u'Python',u'D3.js'])

d.addSeeAlso([
        {u'alias': u'the how it works...', u'link': u'/player/zbbvevhberbv'},
        {u'alias': u'MapReduce with Spark on .csv files', u'link': u'/player/zUCVrbv'}
    ])

#d.addCredits([u'<a href="https://twitter.com/kentenglish">@kentenglish</a> who intruduce to me the legend trick on <a href="http://zeroviscosity.com">zeroviscosity</a>',
#              u'You can find the open data on <a href="http://opendata.paris.fr/explore/dataset/accidentologie/?tab=table">opendata.paris</a>',])
d.date = date(2015, 4, 20)

In [ ]:
d.show

In [ ]:
esConn.index(d.show, 'lmd', 'demo')

#Update 

### find the document

In [ ]:
doc = Demo.fetch('AU8jgcj2gzIYf3juNf_8', esConn)

In [ ]:
d.show

### Perform update

In [ ]:
doc.tools = [u'Spark', u'D3.js']

### Save

In [ ]:
esConn.index(doc.show, u'lmd', u'demo', doc._id)